### Random forest with feature selection - RFE

In [10]:
import pandas as pd
import urllib
from urllib import request
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.preprocessing import label_binarize
import sklearn 
from random import seed
from random import randrange
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import r2_score
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from subprocess import check_output
from datetime import time
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

url = "https://raw.githubusercontent.com/LuisM78/Appliances-energy-prediction-data/master/energydata_complete.csv"
data = pd.read_csv(url)
data['date'] = pd.to_datetime(data['date'])
data['year'], data['month'] , data['time'] , data['hour'] ,data['day'] , data['day_of_week'],data['Numerical_Week'] = data['date'].dt.year, data['date'].dt.month , data['date'].dt.time , data['date'].dt.hour , data['date'].dt.day , data['date'].dt.weekday_name,data['date'].dt.weekday
data['weekStatus'] = data['date'].dt.dayofweek
data['WeekStatus'] = np.where(data['weekStatus'] < 5, 'Weekday', 'Weekend')
data['DateStr'] = data['date'].apply(lambda x: x.strftime('%Y%m%d%H%M'))
data.head(5)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,year,month,time,hour,day,day_of_week,Numerical_Week,weekStatus,WeekStatus,DateStr
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,2016,1,17:00:00,17,11,Monday,0,0,Weekday,201601111700
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,2016,1,17:10:00,17,11,Monday,0,0,Weekday,201601111710
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,2016,1,17:20:00,17,11,Monday,0,0,Weekday,201601111720
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,2016,1,17:30:00,17,11,Monday,0,0,Weekday,201601111730
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,2016,1,17:40:00,17,11,Monday,0,0,Weekday,201601111740


#### Dropping irrelevant columns

In [3]:
data1 = data.drop('date', 1)
data1 = data1.drop('Appliances', 1)
data1 = data1.drop('day_of_week',1)
data1 = data1.drop('weekStatus',1)
data1 = data1.drop('time',1)
data1 = data1.drop('T1',1)
data1 = data1.drop('T4',1)
data1 = data1.drop('T6',1)
data1 = data1.drop('T7',1)
data1 = data1.drop('T_out',1)
data1 = data1.drop('RH_1',1)
data1 = data1.drop('RH_6',1)
data1 = data1.drop('RH_8',1)
data1 = data1.drop('RH_9',1)
data1 = data1.drop('rv1',1)
data1 = data1.drop('rv2',1)
data1 = data1.drop('year',1)
data1 = data1.drop('month',1)
data1 = data1.drop('day',1)
data1 = data1.drop('Numerical_Week',1)
data1 = data1.drop('Windspeed',1)
data1 = data1.drop('Visibility',1)
data1 = data1.drop('WeekStatus',1)

data1.head(5)

,lights,T2,RH_2,T3,RH_3,RH_4,T5,RH_5,RH_7,T8,T9,Press_mm_hg,RH_out,Tdewpoint,hour,DateStr
0,30,19.2,44.790000,19.79,44.730000,45.566667,17.166667,55.20,41.626667,18.2,17.033333,733.5,92.0,5.3,17,201601111700
1,30,19.2,44.722500,19.79,44.790000,45.992500,17.166667,55.20,41.560000,18.2,17.066667,733.6,92.0,5.2,17,201601111710
2,30,19.2,44.626667,19.79,44.933333,45.890000,17.166667,55.09,41.433333,18.2,17.000000,733.7,92.0,5.1,17,201601111720
3,40,19.2,44.590000,19.79,45.000000,45.723333,17.166667,55.09,41.290000,18.1,17.000000,733.8,92.0,5.0,17,201601111730
4,40,19.2,44.530000,19.79,45.000000,45.530000,17.200000,55.09,41.230000,18.1,17.000000,733.9,92.0,4.9,17,201601111740


In [6]:
# defining feature matrix(X) and response vector(y)
X = data1
y = data.Appliances
 
# splitting X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7,
                                                    random_state=1)

pipeline = make_pipeline(preprocessing.MinMaxScaler(), 
                         RandomForestRegressor(n_estimators=100))
# Declare hyperparameters to tune
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
 
clf.fit(X_train, y_train)

# Evaluate model pipeline on test data
pred = clf.predict(X_test)

#### Predicting the target and calculating R sqaure, MAE, RMSE and MAPE

In [ ]:
#Doing predictions on training and test data 
predicted_train=clf1.predict(X_train)
predicted_test=clf1.predict(X_test)
#data_test_predicted=reg.predict(data_test1)

#datapredict_test=reg.predict(data_test1)
print("predicted_train : " + str(predicted_train))
print("predicted_test : " + str(predicted_test))
#print("data_test_predicted : " + str(data_test_predicted))
#print("predicted_test : " + str(datapredict_test))


#### MAE Calculation of model
def mae(actual,prediction):
    return mean_absolute_error(actual,prediction)
test_mae=mae(y_test,predicted_test)
train_mae=mae(y_train,predicted_train)
print("test_mae : " + str(test_mae))
print("train_mae : " + str(train_mae))


#### RMSE Calculation of model
def rmse(actual,prediction):
    return np.sqrt(mean_squared_error(actual,prediction))
test_rmse = rmse(y_test,predicted_test)
train_rmse=rmse(y_train,predicted_train)
print("test_rmse : " + str(test_rmse))
print("train_rmse : " + str(train_rmse))

#### R Squared error calculation
test_r2=r2_score(y_test,predicted_test)
train_r2=clf1.score(X_train,y_train)
print("test_r2 : " + str(test_r2))
print("train_r2 : " + str(train_r2))



#### Calculating MAPE
def mean_absolute_percentage_error(y_test,x_predict):
    np.seterr(divide='ignore',invalid='ignore')
    y_test,x_predict=np.array(y_test),np.array(x_predict) 
    return np.mean(np.abs((y_test - x_predict)/y_test))*100   
test_mape = mean_absolute_percentage_error(y_test, predicted_test)
train_mape = mean_absolute_percentage_error(y_train, predicted_train)
print("test_mape : " + str(test_mape))
print("train_mape : " + str(train_mape))


 
# variance score: 1 means perfect prediction
print('Variance score: {}'.format(clf1.score(X_test, y_test)))